In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [4]:
data = pd.read_excel('mRNA-seq_QC_RKPM.xlsx')

In [6]:
subdata=data[["Sample","Species","Brain","Region","Age","Predicted_age","F2B"]]
regions=["VFC","MFC","DFC","OFC","M1C","S1C","A1C","V1C","IPC","STC","ITC"]
#regions=["M1C","S1C","A1C","V1C"]
subdata=subdata.query("Region in @regions")

In [7]:
macaque=subdata.query('Species=="Macaque"')
human=subdata.query('Species=="Human"')

In [8]:
# Grouper les données par temps et calculer la moyenne et l'écart-type
human_grouped = human.groupby('Predicted_age').agg({'F2B': ['mean', 'sem']})
macaque_grouped = macaque.groupby('Predicted_age').agg({'F2B': [np.mean, np.std]})

# Renommer les colonnes pour une meilleure lisibilité
human_grouped.columns = ['Mean', 'SEM']
macaque_grouped.columns = ['Mean', 'STD']

In [10]:
# Créer une nouvelle colonne 'log2_age' avec les valeurs de Predicted_age en log2
human['log2_age'] = np.log2(human['Predicted_age'])

# Définir les limites des tranches de log2
bins = np.arange(np.floor(human['log2_age'].min()), np.ceil(human['log2_age'].max())+1)

# Regrouper les données par tranche de log2
human_grouped_log2 = human.groupby(pd.cut(human['log2_age'], bins=bins, include_lowest=True)).agg({'F2B': ['mean', 'sem']})

# Supprimer l'index hiérarchique des colonnes
human_grouped_log2.columns = human_grouped_log2.columns.droplevel(0)

# Renommer les colonnes pour une meilleure lisibilité
human_grouped_log2.columns = ['Mean', 'SEM']

# Extraire le premier entier de chaque tranche de log2
human_grouped_log2['log2_age'] = human_grouped_log2.index.astype(str).str.split(',').str[1].str.strip(']')
human_grouped_log2 = human_grouped_log2.reset_index(drop=True)


/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_8193/568711966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human['log2_age'] = np.log2(human['Predicted_age'])


In [12]:
# Créer une nouvelle colonne 'log2_age' avec les valeurs de Predicted_age en log2
macaque['log2_age'] = np.log2(macaque['Predicted_age'])

# Définir les limites des tranches de log2
bins = np.arange(np.floor(macaque['log2_age'].min()), np.ceil(macaque['log2_age'].max())+1)

# Regrouper les données par tranche de log2
macaque_grouped_log2 = macaque.groupby(pd.cut(macaque['log2_age'], bins=bins, include_lowest=True)).agg({'F2B': ['mean', 'sem']})

# Supprimer l'index hiérarchique des colonnes
macaque_grouped_log2.columns = macaque_grouped_log2.columns.droplevel(0)



# Renommer les colonnes pour une meilleure lisibilité
macaque_grouped_log2.columns = ['Mean', 'SEM']

# Extraire le premier entier de chaque tranche de log2
macaque_grouped_log2['log2_age'] = macaque_grouped_log2.index.astype(str).str.split(',').str[1].str.strip(']')
macaque_grouped_log2 = macaque_grouped_log2.reset_index(drop=True)


/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_8193/4069638883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macaque['log2_age'] = np.log2(macaque['Predicted_age'])


In [65]:
# Créer un objet ExcelWriter pour écrire les deux DataFrames dans le même fichier
writer = pd.ExcelWriter('extracted_RNAseq_macaqueVShuman_RKPM.xlsx', engine='xlsxwriter')

# Écrire les deux DataFrames dans des feuilles différentes
human_grouped.to_excel(writer, sheet_name='Human')
macaque_grouped.to_excel(writer, sheet_name='Macaque')

# Sauvegarder le fichier Excel
writer.save()



In [67]:
# Créer un objet ExcelWriter pour écrire les deux DataFrames dans le même fichier
writer_log2 = pd.ExcelWriter('extracted_RNAseq_macaqueVShuman_log2_RKPM.xlsx', engine='xlsxwriter')

# Écrire les deux DataFrames dans des feuilles différentes
human_grouped_log2.to_excel(writer_log2, sheet_name='Human')
macaque_grouped_log2.to_excel(writer_log2, sheet_name='Macaque')

# Sauvegarder le fichier Excel
writer_log2.save()